In [34]:
import pandas as pd
import numpy as np

In [35]:
df = pd.DataFrame([2988.826, 3041.892, 3155.796, 2994.507], [2022, 2021, 2020, 2019])

In [36]:
df.rename(columns={0: 'Investments'}, inplace=True)
df.index.name = 'Date'

In [37]:
#interpolation of days
df.head()

,Investments
Date,
2022,2988.826
2021,3041.892
2020,3155.796
2019,2994.507


In [38]:
monthly_weights = {
    "January": 0.07,  # Coldest month, lowest weight
    "February": 0.07,  # Cold month, low weight
    "March": 0.08,
    "April": 0.08,
    "May": 0.085,
    "June": 0.100,  # Hot month, higher weight
    "July": 0.110,  # Hottest month, highest weight
    "August": 0.100,  # Hot month, high weight
    "September": 0.085,
    "October": 0.080,
    "November": 0.07,
    "December": 0.07  # Cold month, lowest weight
}

In [39]:
for month, weight in monthly_weights.items():
    df[month] = df['Investments'] * weight

In [40]:
df.head()

,Investments,January,February,March,April,May,June,July,August,September,October,November,December
Date,,,,,,,,,,,,,
2022,2988.826,209.21782,209.21782,239.10608,239.10608,254.050210,298.8826,328.77086,298.8826,254.050210,239.10608,209.21782,209.21782
2021,3041.892,212.93244,212.93244,243.35136,243.35136,258.560820,304.1892,334.60812,304.1892,258.560820,243.35136,212.93244,212.93244
2020,3155.796,220.90572,220.90572,252.46368,252.46368,268.242660,315.5796,347.13756,315.5796,268.242660,252.46368,220.90572,220.90572
2019,2994.507,209.61549,209.61549,239.56056,239.56056,254.533095,299.4507,329.39577,299.4507,254.533095,239.56056,209.61549,209.61549


In [41]:
# Number of days per month (non-leap year)
days_per_month = {
    "January": 31, "February": 28, "March": 31, "April": 30,
    "May": 31, "June": 30, "July": 31, "August": 31,
    "September": 30, "October": 31, "November": 30, "December": 31
}

# Adjust the days for leap years
days_per_month_leap = days_per_month.copy()
days_per_month_leap["February"] = 29  # February has 29 days in leap years
days = [days_per_month, days_per_month, days_per_month_leap, days_per_month]

In [42]:
# Create a date range for all days from 2019 to 2022
date_range = pd.date_range(start='2019-01-01', end='2022-12-31', freq='D')

# Create an empty dataframe with the date range as the index
df_empty = pd.DataFrame(index=date_range)
# Add the 'Investments' column to the empty dataframe
df_empty['Investments'] = np.zeros(len(df_empty))

In [43]:
df_empty.head()

,Investments
2019-01-01,0.0
2019-01-02,0.0
2019-01-03,0.0
2019-01-04,0.0
2019-01-05,0.0


In [44]:
list(days_per_month.items())[0][1]

31

In [45]:
df.iloc[0][list(days_per_month.items())[0][0]]

np.float64(209.21782000000002)

In [46]:
# Years
for i in range(4):
    # Months
    for j in range(12):
        # Days
        for k in range(list(days[i].items())[j][1]):
            date = f"{2019 + i}-{j + 1}-{k + 1}"
            df_empty.loc[date, 'Investments'] = df.iloc[i][list(days_per_month.items())[j][0]]/list(days[i].items())[j][1]

In [57]:

df_empty.index.name = 'Date'
df_empty.head()

,Investments
Date,
2019-01-01 00:00:00,6.748962
2019-01-02 00:00:00,6.748962
2019-01-03 00:00:00,6.748962
2019-01-04 00:00:00,6.748962
2019-01-05 00:00:00,6.748962
